In [17]:
import os.path 
import shutil 
import csv 
from binfiles import categorize #vedere commento alla funzione 'get_metadata'
from PIL import Image 
import numpy as np 
from tabulate import tabulate 

# Step 1

Nella cartella *files* ci sono 8 file:
- 2 file di testo
- 2 file audio
- 4 immagini,

con varie estensioni.


Iniziamo creando uno script Python che iteri in ordine alfabetico sui file della cartella files e, a seconda del tipo (audio, documento, immagine), li sposti nella relativa sottocartella (qui sotto trovi un esempio). Se la sottocartella non esiste, lo script la dovrà creare automaticamente.

Durante il ciclo, lo script stamperà le informazioni dei file: nome, tipo e dimensione in byte. Questo è l'output desiderato:

---

La prima funzione serve a ricavare la lista dei file presenti nella directory (ignorando le sotto-cartelle) e a stampare le loro informazioni.  
Questa funzione prende in input una directory e una lista vuota, che verrà riempita dallo script

In [18]:
def get_metadata(directory, file_list):
    #file_names contiene i nomi di tutti i file presenti all'interno della directory, escludendo le cartelle grazie alla condizione if
    file_names = [file for file in sorted(os.listdir(directory)) if (os.path.isfile(os.path.join(directory, file))) and (file!= 'recap.csv')] 
    print('File spostati:')
    for file in file_names:
        tmp = file.split('.') #si separa il nome del file dalla sua estensione
        size = os.path.getsize(directory+'/'+file) #ricaviamo la dimensione in bytes di ogni file
        file_list.append([tmp[0], tmp[1], size]) #aggiungiamo queste tre informazioni alla lista vuota
        file_type = categorize(tmp[1])
        '''
        N.B. 'categorize' è una funzione scaricata da GitHub e contenuta nel file 'binfiles.py'. Serve ad accettare il maggior numero di estensioni possibile.
        Non era strettamente necessaria per questo progetto, ma permette una migliore scalabilità.
        '''
        print(f'Name: {tmp[0]}, Type: {file_type}, Size: {size} Bytes') #stampiamo a schermo il nome del file, il suo tipo e la dimensione

In [19]:
get_metadata('files', [])

File spostati:
Name: bw, Type: image, Size: 94926 Bytes
Name: ciao, Type: doc, Size: 12 Bytes
Name: daffodil, Type: image, Size: 24657 Bytes
Name: eclipse, Type: image, Size: 64243 Bytes
Name: pippo, Type: doc, Size: 8299 Bytes
Name: song1, Type: audio, Size: 1087849 Bytes
Name: song2, Type: audio, Size: 764176 Bytes
Name: trump, Type: image, Size: 10195 Bytes


La seconda funzione crea un file di recap e ci scrive le informazioni contenute in *file_list*

In [20]:
def create_recap(directory, file_list): #ora file_list è stata riempita di informazioni grazie alla funzione precedente
    with open(directory+'/recap.csv', 'w', newline='') as recap:
        writer = csv.writer(recap)
        writer.writerow(['name', 'type', 'size']) #si scrive prima una riga di header
        for i, file in enumerate(file_list):
            type = categorize(file[1]) #si ricava di nuovo il tipo generico di file a partire dalla sua estensione
            writer.writerow([file_list[i][0], type, file_list[i][2]]) #si scrive una nuova riga con le 3 informazioni necessarie

La terza funzione aggiorna il file di recap precedentemente creato. Le uniche due differenze sono queste: 
- il file si apre in modalità *append* anziché *write*
- non si scrive una riga per gli header

In [21]:
def update_recap(directory, file_list):
    with open(directory+'/recap.csv', 'a', newline='') as recap:
        writer = csv.writer(recap)
        for i, file in enumerate(file_list):
            type = categorize(file[1])
            writer.writerow([file_list[i][0], type, file_list[i][2]])
            counter += 1

La quarta funzione muove i file nella cartella indicata. Se questa cartella non esiste, viene creata.

In [22]:
def move(directory, file_list):
    for file in file_list:
        type = categorize(file[1])
        starting_path = f'{directory}/{file[0]}.{file[1]}' #il path di partenza è composto dalla directory, il nome del file (file[0]) e la sua estensione (file[1])
        destination_path = f'{directory}/File {type.capitalize()}' #il path di arrivo è composto dalla directory e dalla cartella 'File +{tipo di file}'
        if not os.path.isdir(destination_path):
            os.mkdir(destination_path) #se la cartella non esiste, viene creata
        shutil.move(starting_path, destination_path) #si muove infine il file dall'origine alla destinazione

Mettiamo tutto assieme con la funzione *main()*:

In [34]:
def main(directory): 
    file_list = []
    get_metadata(directory, file_list)
    if os.path.isfile(directory+'/recap.csv'): #se 'recap.csv' esiste, viene avviata 'update_recap', altrimenti 'create_recap'
        update_recap(directory, file_list)
    else:
        create_recap(directory, file_list)
    move(directory, file_list)

main('files') 

File spostati:
Name: bw, Type: image, Size: 94926 Bytes
Name: ciao, Type: doc, Size: 12 Bytes
Name: daffodil, Type: image, Size: 24657 Bytes
Name: eclipse, Type: image, Size: 64243 Bytes
Name: pippo, Type: doc, Size: 8299 Bytes
Name: song1, Type: audio, Size: 1087849 Bytes
Name: song2, Type: audio, Size: 764176 Bytes
Name: trump, Type: image, Size: 10195 Bytes


# Step 2

Inseriamo lo script creato in un eseguibile chiamato addfile.py dotato di interfaccia a linea di comando (CLI).

Lo scopo dell'eseguibile è spostare un singolo file (che si trova nella cartella files) nella sottocartella di competenza, aggiornando il recap.

L'interfaccia dell'eseguibile ha come unico argomento (obbligatorio) il nome del file da spostare (comprensivo di formato, es: 'ciao.txt'). Nel caso in cui il file passato come argomento non esista, l'interfaccia deve comunicarlo all'utente.

# Step 3

Una immagine in scala di grigio ha un solo livello di colore, una RGB ne ha 3, una RGBA 4 (l'ultimo è detto canale alpha).

Il modulo *Image* della libreria PIL permette di caricare un'immagine, che può essere trasformata in un array NumPy attraverso la funzione np.array. A partire da tale array, è possibile capire se l'immagine caricata è in scala di grigio, RGB o RGBA.

Aggiungiamo al notebook dello Step 1 uno script che iteri sulla sottocartella images e costruisca una tabella riassuntiva prodotta con la libreria *tabulate*.  
Oltre al nome del file, la tabella riporta:

- altezza dell'immagine, in pixel
- larghezza dell'immagine, in pixel
- se l'immagine è in scala di grigio, la colonna *grayscale* indica la media dei valori dell'unico livello di colore
- se l'immagine è a colori, le altre colonne indicano la media dei valori di ogni livello di colore.


In [37]:
def image_processing():
    print('Elenco dei file immagine presenti nella cartella delle immagini:\n')
    #image_names è una lista che recupera i nomi di tutti i file presenti nella cartella delle immagini
    image_names = [image for image in os.listdir('files/File Image') if os.path.isfile(os.path.join('files/File Image',image))] 
    x = len(image_names) #x ci serve per creare la lista iniziale
    #'dict' è un dizionario in cui ogni chiave è associata a tanti zeri quanti sono i file nella cartella. Man mano, questi zeri verranno sostituiti dai valori reali
    dict = {'name':[0]*x, 'height':[0]*x, 'width':[0]*x, 'grayscale':[0]*x, 'R':[0]*x, 'G':[0]*x, 'B':[0]*x, 'ALPHA':[0]*x}
    for i, image in enumerate(sorted(image_names)):
        with Image.open('files/File Image/'+image, mode='r') as im:
            array = np.array(im)
            dict['name'][i] = image.split('.')[0] #aggiungiamo il nome del file al dizionario
            dict['height'][i] = array.shape[0] #aggiungiamo il numero di pixel in altezza (cioè il numero di righe dell'array)
            dict['width'][i] = array.shape[1] #aggiungiamo il numero di pixel in larghezza (cioè il numero di colonne dell'array)
            if len(array.shape) == 2:  #le immagini in bianco e nero non hanno una terza dimensione 
                dict['grayscale'][i] = array.mean().round(2) #mediamo tutti i valori presenti nell'array dell'immagine in bianco e nero
            else: 
                dict['R'][i] = array[:,:,0].mean() #mediamo tutti i valori presenti nella terza dimensione che riguardano il rosso
                dict['G'][i] = array[:,:,1].mean()
                dict['B'][i] = array[:,:,2].mean()
                if array.shape[2] == 4:
                    dict['ALPHA'][i] = array[:,:,3].mean().round(2) #se la terza dimensione dell'array è composta da liste di 4 elementi, modifichiamo anche il valore di alpha
    print(tabulate(dict, headers='keys', floatfmt=".2f")) #usiamo il dizionario 'table' 

In [38]:
image_processing()

Elenco dei file immagine presenti nella cartella delle immagini:

name        height    width    grayscale       R       G      B    ALPHA
--------  --------  -------  -----------  ------  ------  -----  -------
bw             512      512        21.48    0.00    0.00   0.00     0.00
daffodil       500      335         0.00  109.23   85.52   4.77     0.00
eclipse        256      256         0.00  109.05  109.52  39.85   133.59
trump          183      275         0.00   97.01   98.99  90.92     0.00
